# Barnes-Hut Method for N-body Simulation


In [35]:
import numpy as np

all_boxes = []
box_tracker = [[]]
G = 1

class Box():
    def __init__(self, parent, centre, size, particles = [], depth = 0) -> None:
        """Initialises a new Box object
        parent: Box - The parent of the box, ie which box was subdivided to construct this one
        centre: [Float] - The coordinate position of the centre of the box
        size: Float - The distance between parallel sides of the box
        particles: [Particle] A list of the particles contained within that box"""
        self.parent = parent
        self.children = []
        self.centre = centre
        self.size = size
        self.particles = particles
        self.unpartitioned = True
        self.depth = depth
        all_boxes.append(self)
        if depth+1 > len(box_tracker):
            box_tracker.append([])
        #print(depth)
        box_tracker[depth].append(self)

    
    def load_particle(self, particle):
        self.particles.append(particle)
        #print(self)
        #print(self.particles)
        if len(self.particles) > 1:
            if self.children == []:
                self.division_cell()
            else:
                self.place_particle(particle)
                
    def division_cell(self):
        #print(f"Splitting depth {self.depth} box")
        self.unpartitioned = False
        new_size = self.size / 2

        new_centres = []
        centre_alterations = [new_size / 2, -new_size / 2]
        for x_centre_alteration in centre_alterations:
            for y_centre_alteration in centre_alterations:
                for z_centre_alteration in centre_alterations:
                    new_centres.append(np.array([self.centre[0] + x_centre_alteration, self.centre[1] + y_centre_alteration, self.centre[2] + z_centre_alteration]))

        for new_centre in new_centres:
            new_box = Box(self, new_centre, new_size, particles=[], depth=self.depth+1)
            self.children.append(new_box)
            #all_boxes.append(new_box)

        for particle in self.particles:
            self.place_particle(particle)

    def place_particle(self, particle):
        particle_map = "".join(np.where(particle.position > self.centre, "1", "0"))
        particle_to_box_dict = {"111" : 0,
                                "110" : 1,
                                "101" : 2,
                                "100" : 3,
                                "011" : 4,
                                "010" : 5,
                                "001" : 6,
                                "000" : 7}
        placement_box = particle_to_box_dict[particle_map]
        #print(f"adding {particle} to box {placement_box}")
        self.children[placement_box].load_particle(particle)
    
    def find_acceleration_for(self, particle, theta):
        # If this box has a single particle, which is not the subject particle itself, then add the force form this box
        if len(self.particles) == 1 and self.particles != []:
            if self.particles[0] != particle:
                delta_acc = newton_acceleration(self, particle)
                particle.acceleration += delta_acc
        # If this box contains multiple particles and is far away, then apprxomiate this group at its COM
        elif self.size / distance(self, particle) < theta:
            delta_acc = newton_acceleration(self, particle)
            particle.acceleration += delta_acc
        # Else, the box contains many nearby particles and higher precision is required, so analyse children
        else:
            for child in self.children:
                child.find_acceleration_for(particle, theta)

class Particle():
    def __init__(self, position, mass) -> None:
        self.position = position
        self.mass = mass
        self.acceleration = np.array([0.,0.,0.])
        self.velocity = np.array([0.,0.,0.])

def distance(box: Box, particle: Particle):
    #print(particle.position)
    #print(box.com_position)
    return np.linalg.norm(particle.position - box.com_position)

def newton_acceleration(box: Box, particle: Particle):
    direction = (box.com_position - particle.position)/np.linalg.norm(box.com_position - particle.position)
    return G * box.mass / (distance(box, particle) ** 2) * direction




In [36]:

n_particles = 100000
max_mass = 10
box_size = 1000
theta=0.5
initial_positions = np.random.random((n_particles,3)) * box_size
initial_particles = []
for initial_position in initial_positions:
    initial_particles.append(Particle(initial_position, np.random.uniform(0,max_mass)))

root_cell = Box(None, np.array([0.5,0.5,0.5])*box_size, box_size)
#all_boxes.append(root_cell)
#box_tracker.append([root_cell])
print(f"btracker: {len(box_tracker)}")
for i, initial_particle in enumerate(initial_particles):
    #print(i)
    #print(initial_particle.position)
    root_cell.load_particle(initial_particle)

print(len(all_boxes))
#print(box_tracker)
print(len(box_tracker))



btracker: 1
389841
13


In [37]:
deepest_boxes_map = np.array([box.unpartitioned for box in all_boxes])
deepest_boxes = np.array(all_boxes)[deepest_boxes_map]
print(len(deepest_boxes))

for tracker_slice in reversed(box_tracker):
    print(len(tracker_slice))
    for box in tracker_slice:
        if box.unpartitioned:
            if box.particles == []:
                box.mass = 0
                box.com_position = box.centre
            else:
                box.mass = box.particles[0].mass
                box.com_position = box.particles[0].position
        else:
            total_mass = 0
            position_unnormalised = np.array([0.,0.,0.])
            for child_box in box.children:
                total_mass += child_box.mass
                position_unnormalised += child_box.mass * child_box.com_position
            box.com_position = position_unnormalised / total_mass
            box.mass = total_mass
                
print(box_tracker[0][0].com_position)

            

print(len(box_tracker[2][0].particles))

341111
16
56
320
2432
18080
118944
212544
32768
4096
512
64
8
1
[498.16327758 498.12868986 499.69493345]
1568


In [38]:
for particle in root_cell.particles:
    #print(f"new particle {particle}")
    root_cell.find_acceleration_for(particle, theta)
    #print(particle.acceleration)

#print(newton_acceleration(root_cell, root_cell.particles[0]))

KeyboardInterrupt: 

In [ ]:
print(root_cell.particles[0].acceleration)

[-0.0078235  -0.04259836 -0.00590047]
